In [1]:
pip install fedot==0.5.2

/var/folders/lh/sktr_jn11jjgtqxbdlvkrhtm0000gn/T/ipykernel_94530/2328779184.py:5: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  from pandas import datetime


ImportError: Traceback (most recent call last):
  File "/Users/simennaess/Library/Python/3.8/lib/python/site-packages/tensorflow/python/pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "/Users/simennaess/Library/Python/3.8/lib/python/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "/Users/simennaess/Library/Python/3.8/lib/python/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.8/lib/python3.8/imp.py", line 242, in load_module
    return load_dynamic(name, filename, file)
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.8/lib/python3.8/imp.py", line 342, in load_dy

In [ ]:
# Additional imports 
import pandas as pd 
import numpy as np

# Imports for creating plots
import matplotlib.pyplot as plt
from pylab import rcParams
rcParams['figure.figsize'] = 18, 7

# Pipeline and nodes
from fedot.core.pipelines.pipeline import Pipeline
from fedot.core.pipelines.node import PrimaryNode, SecondaryNode

# Data 
from fedot.core.data.data import InputData
from fedot.core.data.data_split import train_test_data_setup
from fedot.core.repository.dataset_types import DataTypesEnum

# Tasks
from fedot.core.repository.tasks import Task, TaskTypesEnum, TsForecastingParams

import logging
logging.raiseExceptions = False

# Read the file

df=pd.read_csv('/Users/simennaess/Documents/Master/Ukentlig_vareid_ys_kunde_time_lag_15_20.csv', parse_dates=['Ukedato'], sep=";", decimal=",", header=0)

#Produkter hentet fra produktseleksjon (vasket og deretter hentet ut basert på at de står for 100%, 50% eller 10% 
#av maksvolumet for enkeltprodukter, samt sesongprodukter. Blant allProducts er det også 12 
#ekstra produkter som ligger mellom mid-> low runners.)

selectedTopProducts=[10133,9662,9399,9306,9400,10135,9898,9630,9897,9860]
selectedMidProducts=[14759,9901,10019,14536,14508,9994,16444,16353,9995, 9307]
selectedLowProducts=[14238,14288,14542,10002,10197,16408,16527, 14563, 9541] 
selectedSeasonalProducts=[10022, 9560, 10005, 9541]

allSelectedProducts=[10133, 9662, 9399, 9306, 9400, 10135, 9898, 9630, 9897, 9860, 14759, 
                     9901, 10019, 14508, 9994, 16444, 16353, 9995, 9205, 14760, 15112, 14536, 16409, 
                     10022, 14540, 14251, 14490, 10017, 9560, 10005, 14238, 10002, 10197, 16408, 14288, 16527, 9541]

df=df[df.Vare_Id.isin(allSelectedProducts)]


In [ ]:
df1= df.groupby(['Ukedato','Vare_Id']).agg({'Sales':'sum'}).reset_index()
df2=df1.loc[df1['Vare_Id']==10133]
        

In [ ]:
# Let's prepare a function for visualizing forecasts
def plot_results(actual_time_series, predicted_values, len_train_data, y_name = 'Parameter'):
    """
    Function for drawing plot with predictions
    
    :param actual_time_series: the entire array with one-dimensional data
    :param predicted_values: array with predicted values
    :param len_train_data: number of elements in the training sample
    :param y_name: name of the y axis
    """
    
    plt.plot(np.arange(0, len(actual_time_series)), 
             actual_time_series, label = 'Actual values', c = 'blue')
    plt.plot(np.arange(len_train_data, len_train_data + len(predicted_values)), 
             predicted_values, label = 'Predicted', c = 'green')
    # Plot black line which divide our array into train and test
    plt.plot([len_train_data, len_train_data],
             [min(actual_time_series), max(actual_time_series)], c = 'orange', linewidth = 1)
    plt.ylabel(y_name, fontsize = 15)
    plt.xlabel('Time index', fontsize = 15)
    plt.legend(fontsize = 15, loc='upper left')
    plt.grid()
    plt.show()

In [ ]:
def multistep_oos_forecast_fedot(df):   
    # Specify forecast length
    horizon = 12

    # Got univariate time series as numpy array
    time_series = np.array(df['Sales'])

    # Wrapp data into InputData
    task = Task(TaskTypesEnum.ts_forecasting,
                TsForecastingParams(forecast_length=horizon))
    input_data = InputData(idx=np.arange(0, len(time_series)),
                           features=time_series,
                           target=time_series,
                           task=task,
                           data_type=DataTypesEnum.ts)

    # Split data into train and test
    train_input, predict_input = train_test_data_setup(input_data)
    pipeline_init_run(train_input, predict_input, horizon)

In [ ]:
def get_pipeline():
    node_lagged_1 = PrimaryNode('lagged')
    node_lagged_1.custom_params = {'window_size': 50}
    node_lagged_2 = PrimaryNode('lagged')
    node_lagged_2.custom_params = {'window_size': 10}

    node_first = SecondaryNode('ridge', nodes_from=[node_lagged_1])
    node_second = SecondaryNode('dtreg', nodes_from=[node_lagged_2])
    node_final = SecondaryNode('ridge', nodes_from=[node_first, node_second])
    pipeline = Pipeline(node_final)

    return pipeline

pipeline = get_pipeline()

In [ ]:
def pipeline_init_run(train_input, predict_input):
    # Fit pipeline
    pipeline.fit(train_input)

    # Make forecast
    output = pipeline.predict(predict_input)
    forecast = np.ravel(np.array(output.predict))

    # Plot the graph
    plot_results(actual_time_series = time_series,
                 predicted_values = forecast, 
                 len_train_data = len(time_series)-forecast_length)